In [1]:
testar_ate = 700

In [2]:
import json

def load_jsonl(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

questoes = load_jsonl('inputs/questoes.jsonl')

questao_por_id = {q['ID_QUESTAO']:q for q in questoes}

In [3]:
import pandas as pd

df = pd.read_json("outputs/4 - respostas_llms/respostas_llms.jsonl", lines=True)

# Filtra apenas os que já tem dados
df = df[df['ID_QUESTAO'] <= testar_ate]

# Converte ID_QUESTAO para string
df["ID_QUESTAO"] = df["ID_QUESTAO"].astype(str)

# Adiciona a resposta correta:
df["RESPOSTA_CORRETA"] = df["ID_QUESTAO"].map(
    lambda id_: questao_por_id[id_]["RESPOSTA"]
)

# Verifica se acertou
df["ACERTOU"] = df["RESPOSTA_CORRETA"] == df["RESPOSTA_LLM"]

In [4]:
import numpy as np

# Agora agrupa e vê o percentual de acerto em cada caso.

# Observação: Se o MODELO_RAG == 'Gold', NUM_CHUNKS deve ser desconsiderado,
# pois ele apenas mostra o total de chunks da base de dados (e que foram enviados ao LLM)
df["NUM_CHUNKS_AGRUP"] = np.where(
    df["MODELO_RAG"] == "Gold",
    "Todos",          # valor fixo para não diferenciar
    df["NUM_CHUNKS"]    # usa o valor real
)

df_resultado = (
    df
    .groupby(
        ["MODELO_LLM", "MODELO_RAG", "TIPO_PESQUISA_CHUNKS", "NUM_CHUNKS_AGRUP"],
        as_index=False
    )["ACERTOU"]
    .mean()
)

df_resultado

,MODELO_LLM,MODELO_RAG,TIPO_PESQUISA_CHUNKS,NUM_CHUNKS_AGRUP,ACERTOU
0,deepseek-chat,,,0,0.877143
1,deepseek-chat,Gold,apenas_art,Todos,0.932857
2,deepseek-chat,Gold,todos_chunks,Todos,0.951429
3,deepseek-chat,bm25,apenas_art,1,0.927143
4,deepseek-chat,bm25,apenas_art,2,0.934286
5,deepseek-chat,bm25,apenas_art,3,0.937143
6,deepseek-chat,bm25,todos_chunks,1,0.921429
7,deepseek-chat,bm25,todos_chunks,2,0.930000
8,deepseek-chat,bm25,todos_chunks,3,0.931429
9,deepseek-chat,text-embedding-3-large,apenas_art,1,0.921429
